In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
from tensorflow.keras import layers

# Load the dataset
# Assuming the dataset is in CSV format, replace 'your_dataset.csv' with the actual file path.
df = pd.read_csv('/content/data.csv')


In [ ]:
df.head()

,First Term Gpa,Second Term Gpa,First Language,Funding,School,FastTrack,Coop,Residency,Gender,Previous Education,Age Group,High School Average Mark,Math Score,English Score,"FirstYearPersistence {1,0}"
0,0,0,1,2,6,2,1,1,2,1,1,59,16,7,1
1,2.5,2,3,4,6,1,2,2,2,1,3,?,?,7,1
2,4.25,3.923077,1,1,6,2,1,1,1,2,3,92,41,9,1
3,3.020833,2.321429,3,4,6,1,2,2,2,2,3,?,?,8,1
4,4.275,4.326923,1,2,6,1,1,1,1,2,3,97,?,9,1


In [ ]:
df.columns

Index(['First Term Gpa', 'Second Term Gpa', 'First Language', 'Funding ',
       'School', 'FastTrack', 'Coop', ' Residency', 'Gender ',
       'Previous Education', 'Age Group', 'High School Average Mark',
       'Math Score', 'English Score', 'FirstYearPersistence {1,0}'],
      dtype='object')

In [ ]:
df.isnull().sum()

,0
First Term Gpa,0
Second Term Gpa,0
First Language,0
Funding,0
School,0
FastTrack,0
Coop,0
Residency,0
Gender,0
Previous Education,0


In [ ]:
# Replace all the '?' and " " by nan
df = df.replace('?', pd.NA)
df = df.replace(' ', pd.NA)

In [ ]:

# Split dataset into features (X) and target (y)
X = df.drop('FirstYearPersistence {1,0}', axis=1)
y = df['FirstYearPersistence {1,0}']

In [ ]:
#Normalize the data using one hot encoding for nan

X = pd.get_dummies(X)
X.fillna(X.mean(), inplace=True)


# Normalize the data

scaler = StandardScaler()
X = scaler.fit_transform(X)


In [ ]:
# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42)


In [ ]:
# Building a CNN

# Reshape the data for CNN (assuming 1D data)
X_train_cnn = X_train.reshape(X_train.shape[0], X_train.shape[1], 1)
X_test_cnn = X_test.reshape(X_test.shape[0], X_test.shape[1], 1)

# Build the CNN model
model_cnn = tf.keras.Sequential([
    layers.Conv1D(32, 3, activation='relu', input_shape=(X_train_cnn.shape[1], 1)),
    layers.MaxPooling1D(2),
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    # Assuming regression for persistence count
    layers.Dense(1, activation='linear')
])

model_cnn.compile(optimizer='adam', loss='mse', metrics=['mae'])


/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [ ]:
# Train the model
model.fit(X_train, y_train, validation_data=(
    X_test, y_test), epochs=10, batch_size=32)

Epoch 1/10
36/36 ━━━━━━━━━━━━━━━━━━━━ 2s 9ms/step - loss: 2.4106 - mae: 1.2520 - val_loss: 0.8258 - val_mae: 0.7232
Epoch 2/10
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.6833 - mae: 0.6073 - val_loss: 0.4688 - val_mae: 0.5423
Epoch 3/10
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.1588 - mae: 0.3089 - val_loss: 0.4280 - val_mae: 0.5076
Epoch 4/10
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0657 - mae: 0.1811 - val_loss: 0.4142 - val_mae: 0.5010
Epoch 5/10
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.0337 - mae: 0.1234 - val_loss: 0.4135 - val_mae: 0.4959
Epoch 6/10
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.0276 - mae: 0.0993 - val_loss: 0.4032 - val_mae: 0.4933
Epoch 7/10
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0202 - mae: 0.0839 - val_loss: 0.4017 - val_mae: 0.4878
Epoch 8/10
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.0199 - mae: 0.0752 - val_loss: 0.3998 - val_mae: 0.4876
Epoch 9/10
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0154 - mae: 

In [ ]:
# Calculate metrics for training set
train_accuracy = accuracy_score(y_train, y_train_pred_class)
train_f1 = f1_score(y_train, y_train_pred_class, average='macro') # Use 'macro' averaging for multiclass
train_precision = precision_score(y_train, y_train_pred_class, average='macro') # Use 'macro' averaging for multiclass
train_recall = recall_score(y_train, y_train_pred_class, average='macro') # Use 'macro' averaging for multiclass

# Calculate metrics for test set
test_accuracy = accuracy_score(y_test, y_test_pred_class)
test_f1 = f1_score(y_test, y_test_pred_class, average='macro') # Use 'macro' averaging for multiclass
test_precision = precision_score(y_test, y_test_pred_class, average='macro') # Use 'macro' averaging for multiclass
test_recall = recall_score(y_test, y_test_pred_class, average='macro') # Use 'macro' averaging for multiclass

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:


# the model summary
model_cnn.summary()

# If you want to print the calculated metrics:
print("Training Accuracy:", train_accuracy)
print("Training F1 Score:", train_f1)
print("Training Precision:", train_precision)
print("Training Recall:", train_recall)

print("Test Accuracy:", test_accuracy)
print("Test F1 Score:", test_f1)
print("Test Precision:", test_precision)
print("Test Recall:", test_recall)


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv1d (Conv1D)                      │ (None, 1542, 32)            │             128 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling1d (MaxPooling1D)         │ (None, 771, 32)             │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 24672)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ (None, 64)                  │       1,579,072 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_4 (Dense)                      │ (None, 1)                   │              65 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 1,579,265 (6.02 MB)

 Trainable params: 1,579,265 (6.02 MB)

 Non-trainable params: 0 (0.00 B)

Training Accuracy: 0.206266318537859
Training F1 Score: 0.12431920780544632
Training Precision: 0.2984872270586556
Training Recall: 0.32392014003879455
Test Accuracy: 0.24305555555555555
Test F1 Score: 0.14182648638829057
Test Precision: 0.31775210084033617
Test Recall: 0.32162001191185224


In [ ]:
# Oversampling with Smote

from imblearn.over_sampling import SMOTE

# Apply SMOTE to oversample the minority class
smote = SMOTE(random_state=42)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)

# Now you can use X_train_resampled and y_train_resampled to train your model
# For example, with the CNN model:
model_cnn.fit(X_train_resampled, y_train_resampled, validation_data=(
    X_test, y_test), epochs=10, batch_size=32)
model_cnn.summary()


Epoch 1/10
58/58 ━━━━━━━━━━━━━━━━━━━━ 2s 41ms/step - loss: 0.0147 - mae: 0.0685 - val_loss: 0.2080 - val_mae: 0.3355
Epoch 2/10
58/58 ━━━━━━━━━━━━━━━━━━━━ 3s 49ms/step - loss: 0.0116 - mae: 0.0597 - val_loss: 0.2059 - val_mae: 0.3353
Epoch 3/10
58/58 ━━━━━━━━━━━━━━━━━━━━ 4s 60ms/step - loss: 0.0099 - mae: 0.0558 - val_loss: 0.2077 - val_mae: 0.3425
Epoch 4/10
58/58 ━━━━━━━━━━━━━━━━━━━━ 4s 40ms/step - loss: 0.0090 - mae: 0.0539 - val_loss: 0.2138 - val_mae: 0.3451
Epoch 5/10
58/58 ━━━━━━━━━━━━━━━━━━━━ 2s 40ms/step - loss: 0.0120 - mae: 0.0582 - val_loss: 0.2143 - val_mae: 0.3447
Epoch 6/10
58/58 ━━━━━━━━━━━━━━━━━━━━ 3s 41ms/step - loss: 0.0105 - mae: 0.0594 - val_loss: 0.2103 - val_mae: 0.3467
Epoch 7/10
58/58 ━━━━━━━━━━━━━━━━━━━━ 4s 67ms/step - loss: 0.0112 - mae: 0.0593 - val_loss: 0.2241 - val_mae: 0.3468
Epoch 8/10
58/58 ━━━━━━━━━━━━━━━━━━━━ 3s 46ms/step - loss: 0.0116 - mae: 0.0594 - val_loss: 0.2162 - val_mae: 0.3514
Epoch 9/10
58/58 ━━━━━━━━━━━━━━━━━━━━ 5s 40ms/step - loss: 0.010

In [ ]:
# Calculating the Accuracy, F1 score, Precision and Recall based on the resampled train and test sets

from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score

X_train_resampled = X_train_resampled.reshape(X_train_resampled.shape[0], X_train_resampled.shape[1], 1)

# Predict on resampled training set
y_train_resampled_pred = model_cnn.predict(X_train_resampled)
y_train_resampled_pred_class = (y_train_resampled_pred > 0.5).astype(int)  # Assuming binary classification

# Predict on test set
y_test_pred = model_cnn.predict(X_test_cnn)
y_test_pred_class = (y_test_pred > 0.5).astype(int)  # Assuming binary classification

# Calculate metrics for resampled training set
train_accuracy_resampled = accuracy_score(y_train_resampled, y_train_resampled_pred_class)
train_f1_resampled = f1_score(y_train_resampled, y_train_resampled_pred_class)
train_precision_resampled = precision_score(y_train_resampled, y_train_resampled_pred_class)
train_recall_resampled = recall_score(y_train_resampled, y_train_resampled_pred_class)

# Calculate metrics for test set (using predictions on original test set)
test_accuracy = accuracy_score(y_test, y_test_pred_class)
test_f1 = f1_score(y_test, y_test_pred_class)
test_precision = precision_score(y_test, y_test_pred_class)
test_recall = recall_score(y_test, y_test_pred_class)

# Print the calculated metrics
print("Resampled Training Accuracy:", train_accuracy_resampled)
print("Resampled Training F1 Score:", train_f1_resampled)
print("Resampled Training Precision:", train_precision_resampled)
print("Resampled Training Recall:", train_recall_resampled)

print("\nTest Accuracy:", test_accuracy)
print("Test F1 Score:", test_f1)
print("Test Precision:", test_precision)
print("Test Recall:", test_recall)


58/58 ━━━━━━━━━━━━━━━━━━━━ 1s 22ms/step
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
Resampled Training Accuracy: 0.9972796517954298
Resampled Training F1 Score: 0.9972722313147845
Resampled Training Precision: 1.0
Resampled Training Recall: 0.9945593035908596

Test Accuracy: 0.7916666666666666
Test F1 Score: 0.8591549295774648
Test Precision: 0.8840579710144928
Test Recall: 0.8356164383561644


In [ ]:
import pickle
import tensorflow as tf
from imblearn.over_sampling import SMOTE
from tensorflow.keras import layers, models

# Define the CNN model architecture
model_cnn = models.Sequential([
    layers.Conv1D(32, kernel_size=3, activation='relu', input_shape=(X_train_resampled.shape[1], 1)), # Use the shape of resampled data
    layers.MaxPooling1D(pool_size=2),
    layers.Conv1D(64, kernel_size=3, activation='relu'),
    layers.MaxPooling1D(pool_size=2),
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dense(1, activation='sigmoid')
])

model_cnn.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Example data loading and preprocessing (replace with actual data)


# Apply SMOTE
smote = SMOTE(random_state=42)
# X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)

# Train the CNN model (replace with your actual training call)
model_cnn.fit(X_train_resampled, y_train_resampled, validation_data=(X_test, y_test), epochs=10, batch_size=32)

# Save the model to an h5 file
model_cnn.save('cnn_model.h5')

# Save the SMOTE object to a pkl file
with open('smote.pkl', 'wb') as f:
    pickle.dump(smote, f)


Epoch 1/10
58/58 ━━━━━━━━━━━━━━━━━━━━ 8s 107ms/step - accuracy: 0.7640 - loss: 0.4966 - val_accuracy: 0.8611 - val_loss: 0.5133
Epoch 2/10
58/58 ━━━━━━━━━━━━━━━━━━━━ 5s 85ms/step - accuracy: 0.9683 - loss: 0.0974 - val_accuracy: 0.8507 - val_loss: 0.7406
Epoch 3/10
58/58 ━━━━━━━━━━━━━━━━━━━━ 5s 92ms/step - accuracy: 0.9917 - loss: 0.0355 - val_accuracy: 0.8611 - val_loss: 0.9177
Epoch 4/10
58/58 ━━━━━━━━━━━━━━━━━━━━ 10s 80ms/step - accuracy: 0.9952 - loss: 0.0160 - val_accuracy: 0.8472 - val_loss: 1.1616
Epoch 5/10
58/58 ━━━━━━━━━━━━━━━━━━━━ 6s 106ms/step - accuracy: 0.9975 - loss: 0.0108 - val_accuracy: 0.8368 - val_loss: 1.2821
Epoch 6/10
58/58 ━━━━━━━━━━━━━━━━━━━━ 8s 76ms/step - accuracy: 0.9990 - loss: 0.0054 - val_accuracy: 0.8299 - val_loss: 1.4011
Epoch 7/10
58/58 ━━━━━━━━━━━━━━━━━━━━ 6s 99ms/step - accuracy: 0.9983 - loss: 0.0047 - val_accuracy: 0.8368 - val_loss: 1.5787
Epoch 8/10
58/58 ━━━━━━━━━━━━━━━━━━━━ 4s 75ms/step - accuracy: 0.9990 - loss: 0.0028 - val_accuracy: 0.8299 